# ANN for Image classification

In [4]:
import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot
import pickle
import torch.optim as op
from torch.utils.data import DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



## Load the CIFAR-10 dataset

In [5]:
def load_cifar_batch(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        images = data[b'data']
        labels = data[b'labels']
        images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape and rearrange dimensions
        return images, labels

def load_cifar10_data(data_dir):
    x_train, y_train = [], []

    for batch_id in range(1,6):  # Load data from batches 1 to 5
        batch_path = f'{data_dir}/data_batch_{batch_id}'
        images, labels = load_cifar_batch(batch_path)
        x_train.append(images)
        y_train.append(labels)

    x_train = np.concatenate(x_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)

    test_batch_path = f'{data_dir}/test_batch'
    x_test, y_test = load_cifar_batch(test_batch_path)

    return x_train, y_train, x_test, y_test

data_dir = 'cifar-10'  # Path to the extracted CIFAR-10 dataset
x_train, y_train, x_test, y_test = load_cifar10_data(data_dir)



## Convert into torch tensors

In [6]:
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
trainSet = (x_train,y_train)
testSet = (x_test,y_test)

## Designing the model

### Init the model inputs, output and layer sizes

In [7]:
inputSize = 32 * 32 * 3 # Every image is 32*32 pixels with 3 RGB channels
# sizes for each hidden layer of the neural netowrk
hiddenSize1 = 4096
hiddenSize2 = 2048
hiddenSize3 = 512
outputSize = 10 # cifar-10 has 10 classes


### Defining the class to develop the neural network

In [8]:
class ANN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(ANN, self).__init__()
        # Define the layers
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, output_size)
        
    def forward(self, x):
        # Apply the layers sequentially
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

### Calling the model with our correct data sizes

In [9]:
model = ANN(inputSize,hiddenSize1,hiddenSize2,hiddenSize3,outputSize)
print(model)

ANN(
  (fc1): Linear(in_features=3072, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=10, bias=True)
)


### Loss function and optimizer

In [10]:
epochs = 10
batchSize = 64
lossFunction = nn.CrossEntropyLoss()
optimizer = op.SGD(model.parameters(), lr=0.001)
trainLoader = DataLoader(trainSet, batch_size=64, shuffle=True)

## Training the model

In [11]:
n_epochs = 10
batch_size = 64
for epoch in range(n_epochs):
    for i in range(0, len(x_train), batch_size):
        Xbatch = x_train[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (65536x3 and 3072x4096)